# Part 5: Deploying the Embedding Lookup and ScaNN Index to AI Platform

This tutorial shows how to use Matrix Factorization algorithm in BigQuery ML to generate embeddings for items based on their cooccurrence statistics. The generated item embeddings can be then used to find similar items.

Part 5 covers the following steps:
1. Deploying the Embedding Lookup SavedModel to AI Platform Prediction. 
2. Deploying the ScaNN index to AI Platform Prediction, using a Custom Container, for real-time similar item matching, which works as follows:
    1. Accepts a query item Id.
    2. Looks up the embedding of the query item Id from Embedding Lookup Model in AI Platform Prediction.
    3. Uses the ScaNN index to find similar item Ids for the given query item embedding.
    4. Returns a list of the similar item Ids to the query item Id.
3. (Optional) Exporting and deploying the BigQuery ML matrix Factorization model to AI Platform for exact matching.


## Setup

### Import libraries

In [ ]:
import numpy as np
import tensorflow as tf

### Configure GCP environment settings

In [ ]:
PROJECT_ID = 'ksalama-cloudml' # Change to your project.
BUCKET = 'ksalama-cloudml' # Change to your bucket.
REGION = 'europe-west2' # Change to your AI Platform Prediction region.
ARTIFACTS_REPOSITORY_NAME = 'ml-serving'

EMBEDDNIG_LOOKUP_MODEL_OUTPUT_DIR = f'gs://{BUCKET}/bqml/embedding_lookup_model'
EMBEDDNIG_LOOKUP_MODEL_NAME = 'item_embedding_lookup'
EMBEDDNIG_LOOKUP_MODEL_VERSION = 'v1'

INDEX_DIR = f'gs://{BUCKET}/bqml/scann_index'
SCANN_MODEL_NAME = 'index_server'
SCANN_MODEL_VERSION = 'v1'

KIND = 'song'

!gcloud config set project $PROJECT_ID

### Authenticate your GCP account
This is required if you run the notebook in Colab

In [ ]:
try:
  from google.colab import auth
  auth.authenticate_user()
  print("Colab user is authenticated.")
except: pass

## Serve an Embedding Lookup Model on AI Platform Prediction

In [ ]:
!gcloud ai-platform models create {EMBEDDNIG_LOOKUP_MODEL_NAME} --region={REGION}

In [ ]:
!gcloud ai-platform versions create {EMBEDDNIG_LOOKUP_MODEL_VERSION} \
  --region={REGION} \
  --model={EMBEDDNIG_LOOKUP_MODEL_VERSION} \
  --origin={EMBEDDNIG_LOOKUP_MODEL_OUTPUT_DIR} \
  --runtime-version=2.2 \
  --framework=TensorFlow \
  --python-version=3.7 \
  --machine-type=n1-standard-2

print("The model version is deployed to AI Platform Prediciton.")

### Test the deployed embedding lookup AI Platform Prediction model

In [ ]:
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions

api_endpoint = f'https://{REGION}-ml.googleapis.com'
client_options = ClientOptions(api_endpoint=api_endpoint)
service = googleapiclient.discovery.build(
    serviceName='ml', version='v1', client_options=client_options)

In [ ]:
def caip_embedding_lookup(input_items):
  request_body = {'instances': input_items}
  service_name = f'projects/{PROJECT_ID}/models/{EMBEDDNIG_LOOKUP_MODEL_NAME}/versions/{EMBEDDNIG_LOOKUP_MODEL_VERSION}'
  print(f'Calling : {service_name}')
  response = service.projects().predict(
    name=service_name, body=request_body).execute()

  if 'error' in response:
    raise RuntimeError(response['error'])

  return response['predictions']

In [ ]:
input_items = ['2114406', '2114402 2120788', 'abc123']

embeddings = caip_embedding_lookup(input_items)
print(f'Embeddings retrieved: {len(embeddings)}')
for idx, embedding in enumerate(embeddings):
  print(f'{input_items[idx]}: {embedding[:5]}')

## Build a Custom Prediction Container for the ScaNN Index
The custom container runs a [gunicorn](https://gunicorn.org/) web server hosting a [Flask](https://flask.palletsprojects.com/en/1.1.x/quickstart/) application. The app loads the ScaNN index to use it for similar items matching.



### Create an Artifact Registry for the Docker container images.

In [ ]:
!gcloud beta artifacts repositories create {ARTIFACTS_REPOSITORY_NAME} \
  --location={REGION} \
  --repository-format=docker

In [ ]:
!gcloud beta auth configure-docker {REGION}-docker.pkg.dev --quiet

### Use Cloud Build to build the Docker container image

You need to enable the `Compute Engine` service account in [Cloud Build setting](https://console.cloud.google.com/cloud-build/settings/service-account).

In [ ]:
IMAGE_URL = f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACTS_REPOSITORY_NAME}/{SCANN_MODEL_NAME}:{SCANN_MODEL_VERSION}'
PORT=5001

SUBSTITUTIONS = ''
SUBSTITUTIONS += f'_IMAGE_URL={IMAGE_URL},'
SUBSTITUTIONS += f'_PORT={PORT}'

!gcloud builds submit --config=index_server/cloudbuild.yaml \
  --substitutions={SUBSTITUTIONS} \
  --timeout=1h

In [ ]:
repository_id = f'{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACTS_REPOSITORY_NAME}'

!gcloud beta artifacts docker images list {repository_id}

## Create a Service Account for the Container to Access Cloud Resources

In [ ]:
SERVICE_ACCOUNT_NAME = 'caip-serving'
SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com'
!gcloud iam service-accounts create {SERVICE_ACCOUNT_NAME} \
  --description="Service account for AI Platform Prediction to access cloud resources." 

We need to grant the Cloud ML Engine (AI Platform) service account the **iam.serviceAccountAdmin** privileges, and grant the new service account the privileges required by the ScaNN matching service: **storage.objectViewer** and **ml.developer**.

In [ ]:
!gcloud projects describe {PROJECT_ID} --format="value(projectNumber)"

In [ ]:
PROJECT_NUMBER=900786220115 # Change to your project number

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --role=roles/iam.serviceAccountAdmin \
  --member=serviceAccount:service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --role=roles/storage.objectViewer \
  --member=serviceAccount:{SERVICE_ACCOUNT_EMAIL}
    
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --role=roles/ml.developer \
  --member=serviceAccount:{SERVICE_ACCOUNT_EMAIL}

## Serve the ScaNN Index Custom Container on AI Platform Prediction

In [ ]:
!gcloud ai-platform models create {SCANN_MODEL_NAME} --region={REGION}

In [ ]:
HEALTH_ROUTE=f'/v1/models/{SCANN_MODEL_NAME}/versions/{SCANN_MODEL_VERSION}'
PREDICT_ROUTE=f'/v1/models/{SCANN_MODEL_NAME}/versions/{SCANN_MODEL_VERSION}:predict'

ENV_VARIABLES = f'PROJECT_ID={PROJECT_ID},'
ENV_VARIABLES += f'REGION={REGION},'
ENV_VARIABLES += f'INDEX_DIR={INDEX_DIR},'
ENV_VARIABLES += f'EMBEDDNIG_LOOKUP_MODEL_NAME={EMBEDDNIG_LOOKUP_MODEL_NAME},'
ENV_VARIABLES += f'EMBEDDNIG_LOOKUP_MODEL_VERSION={EMBEDDNIG_LOOKUP_MODEL_VERSION}'

!gcloud beta ai-platform versions create {SCANN_MODEL_VERSION} \
  --region={REGION} \
  --model={SCANN_MODEL_NAME} \
  --image={IMAGE_URL} \
  --ports={PORT} \
  --predict-route={PREDICT_ROUTE} \
  --health-route={HEALTH_ROUTE} \
  --machine-type=n1-standard-4 \
  --env-vars={ENV_VARIABLES} \
  --service-account={SERVICE_ACCOUNT_EMAIL}

print("The model version is deployed to AI Platform Prediction.")

## Test the Deployed ScaNN Index Service

In [ ]:
from google.cloud import datastore
import requests
client = datastore.Client(PROJECT_ID)

In [ ]:
def caip_scann_match(query_items, show=10):
  request_body = {
      'instances': [{
          'query':' '.join(query_items), 
          'show':show
      }]
   }
  
  service_name = f'projects/{PROJECT_ID}/models/{SCANN_MODEL_NAME}/versions/{SCANN_MODEL_VERSION}'
  print(f'Calling: {service_name}')  
  response = service.projects().predict(
    name=service_name, body=request_body).execute()

  if 'error' in response:
    raise RuntimeError(response['error'])

  match_tokens = response['predictions']
  keys = [client.key(KIND, int(key)) for key in match_tokens]
  items = client.get_multi(keys)
  return items


In [ ]:
songs = {
    '2114406': 'Metallica: Nothing Else Matters',
    '2114402': 'Metallica: The Unforgiven',
    '2120788': 'Limp Bizkit: My Way',
    '2120786': 'Limp Bizkit: My Generation',
    '1086322': 'Jacques Brel: Ne Me Quitte Pas',
    '3129954': 'Édith Piaf: Non, Je Ne Regrette Rien',
    '53448': 'France Gall: Ella, Elle l\'a',
    '887688': 'Enrique Iglesias: Tired Of Being Sorry',
    '562487': 'Shakira: Hips Don\'t Lie',
    '833391': 'Ricky Martin: Livin\' la Vida Loca',
    '1098069': 'Snoop Dogg: Drop It Like It\'s Hot',
    '910683': '2Pac: California Love',
    '1579481': 'Dr. Dre: The Next Episode',
    '2675403': 'Eminem: Lose Yourself',
    '2954929': 'Black Sabbath: Iron Man',
    '625169': 'Black Sabbath: Paranoid',
}

In [ ]:
for item_Id, desc in songs.items():
  print(desc)
  print("==================")
  similar_items = caip_scann_match([item_Id], 5)
  for similar_item in similar_items:
    print(f'- {similar_item["artist"]}: {similar_item["track_title"]}')
  print()

## (Optional) Export and Deploy the BigQuery Model to AI Platform

### Import the model from BigQuery ML to Cloud Storage as a SavedModel
This will export the matrix factorization as an exact matching model. The model takes Item1_Id as an input and outputs the top 50 recommended item2_Ids.

In [ ]:
BQ_DATASET_NAME = 'recommendations'
BQML_MODEL_NAME = 'item_matching_model'
BQML_MODEL_VERSION = 'v1' 
BQML_MODEL_OUTPUT_DIR = f'gs://{BUCKET}/bqml/item_matching_model'

!bq --quiet extract -m {BQ_DATASET_NAME}.{BQML_MODEL_NAME} {BQML_MODEL_OUTPUT_DIR}

In [ ]:
!saved_model_cli show --dir {BQML_MODEL_OUTPUT_DIR} --tag_set serve --signature_def serving_default

### Deploy the item matching SavedModel to AI Platform Prediction

In [ ]:
!gcloud ai-platform models create {BQML_MODEL_NAME} --region={REGION}

In [ ]:
!gcloud ai-platform versions create {MODEL_VERSION} \
  --region={REGION} \
  --model={BQML_MODEL_NAME} \
  --origin={BQML_MODEL_OUTPUT_DIR} \
  --runtime-version=2.2 \
  --framework=TensorFlow \
  --python-version=3.7 \
  --machine-type=n1-standard-2

print("The model version is deployed to AI Platform Prediciton.")

In [ ]:
def caip_bqml_matching(input_items, show):
  request_body = {'instances': input_items}
  service_name = f'projects/{PROJECT_ID}/models/{BQML_MODEL_NAME}/versions/{BQML_MODEL_VERSION}'
  print(f'Calling : {service_name}')
  response = service.projects().predict(
    name=service_name, body=request_body).execute()

  if 'error' in response:
    raise RuntimeError(response['error'])
    
  
  match_tokens = response['predictions'][0]["predicted_item2_Id"][:show]
  keys = [client.key(KIND, int(key)) for key in match_tokens]
  items = client.get_multi(keys)
  return items

  return response['predictions']

In [ ]:
for item_Id, desc in songs.items():
  print(desc)
  print("==================")
  similar_items = caip_bqml_matching([int(item_Id)], 5)
  for similar_item in similar_items:
    print(f'- {similar_item["artist"]}: {similar_item["track_title"]}')
  print()

## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**